# Advanced Research Features Tutorial

This notebook demonstrates the cutting-edge research capabilities in IRST Library, including:
- Quantum-inspired neural networks
- Physics-informed neural networks
- Continual learning methods
- Adversarial robustness
- Synthetic data generation

These features represent the state-of-the-art in infrared small target detection research.

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

# Import IRST Library research modules
from irst_library.research import (
    # Quantum Neural Networks
    create_quantum_irst_model,
    QuantumInspiredLoss,
    
    # Physics-Informed Networks
    create_physics_informed_model,
    PhysicsInformedLoss,
    
    # Continual Learning
    create_continual_learning_setup,
    ContinualLearningTrainer,
    
    # Adversarial Robustness
    create_attack_suite,
    RobustnessEvaluator,
    
    # Synthetic Data
    create_synthetic_dataset,
    SyntheticDataConfig
)

print("🚀 Advanced IRST Library Research Features Loaded!")

## 1. Quantum-Inspired Neural Networks

Explore quantum computing principles applied to infrared target detection.

In [ ]:
# Create a quantum-inspired hybrid model
quantum_model = create_quantum_irst_model(
    model_type='hybrid',
    input_channels=1,
    num_classes=2,
    classical_features=256,
    quantum_qubits=8
)

print(f"✨ Quantum Model Architecture:")
print(quantum_model)

# Create quantum-inspired loss function
quantum_loss = QuantumInspiredLoss(alpha=0.7, beta=0.3)

# Test forward pass
dummy_input = torch.randn(4, 1, 64, 64)
dummy_targets = torch.randint(0, 2, (4,))

with torch.no_grad():
    outputs = quantum_model(dummy_input)
    loss_dict = quantum_loss(
        outputs['logits'], 
        dummy_targets, 
        outputs['quantum_output']
    )

print(f"\n🔮 Quantum Model Output Keys: {list(outputs.keys())}")
print(f"🔮 Quantum Loss Components: {list(loss_dict.keys())}")
print(f"🔮 Total Loss: {loss_dict['total_loss']:.4f}")

## 2. Physics-Informed Neural Networks

Integrate physical laws and constraints into neural network training.

In [ ]:
# Create physics-informed model
physics_model = create_physics_informed_model(
    model_type='standard',
    input_channels=1,
    num_classes=2,
    physics_laws=['atmospheric', 'heat_transfer', 'infrared'],
    predict_physics=True
)

print(f"🌡️ Physics-Informed Model:")
print(f"Number of physics laws: {len(physics_model.physics_laws)}")

# Create physics-informed loss
physics_loss_fn = PhysicsInformedLoss(
    physics_loss_weight=0.2,
    adaptive_weighting=True
)

# Test physics predictions
dummy_coords = torch.rand(4, 2)  # x, y coordinates

with torch.no_grad():
    physics_outputs = physics_model(dummy_input, coordinates=dummy_coords)
    physics_losses = physics_model.compute_physics_loss(
        dummy_input, physics_outputs, coordinates=dummy_coords
    )
    
    total_loss = physics_loss_fn(
        physics_outputs, dummy_targets, physics_losses
    )

print(f"\n🌡️ Physics Output Keys: {list(physics_outputs.keys())}")
print(f"🌡️ Physics Loss Keys: {list(physics_losses.keys())}")
print(f"🌡️ Temperature Range: {physics_outputs['temperature'].min():.1f}K - {physics_outputs['temperature'].max():.1f}K")
print(f"🌡️ Total Physics Loss: {total_loss['total_loss']:.4f}")

## 3. Continual Learning

Learn new tasks without forgetting previous knowledge using Elastic Weight Consolidation.

In [ ]:
# Create base model for continual learning
base_model = nn.Sequential(
    nn.Conv2d(1, 32, 3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(32, 64, 3, padding=1),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d((4, 4)),
    nn.Flatten(),
    nn.Linear(64 * 4 * 4, 2)
)

# Setup continual learning with EWC
continual_strategy, replay_buffer = create_continual_learning_setup(
    base_model=base_model,
    strategy='ewc',
    strategy_params={
        'lambda_ewc': 1000.0,
        'fisher_estimation_samples': 100
    },
    use_replay=True,
    replay_params={
        'buffer_size': 1000,
        'selection_strategy': 'gradient_episodic'
    }
)

# Create continual learning trainer
continual_trainer = ContinualLearningTrainer(
    model=base_model,
    continual_strategy=continual_strategy,
    replay_buffer=replay_buffer
)

print(f"🧠 Continual Learning Setup:")
print(f"Strategy: {continual_strategy.__class__.__name__}")
print(f"Replay Buffer Size: {replay_buffer.buffer_size}")
print(f"Selection Strategy: {replay_buffer.selection_strategy}")

# Simulate adding samples to replay buffer
replay_buffer.add_samples(
    dummy_input, dummy_targets, task_id=0, model=base_model
)

print(f"\n🧠 Replay Buffer Status:")
print(f"Current size: {replay_buffer.current_size}")
print(f"Samples added successfully!")

## 4. Adversarial Robustness

Evaluate and improve model robustness against adversarial attacks.

In [ ]:
# Create adversarial attack suite
attack_suite = create_attack_suite(
    epsilon=0.1,
    norm='inf',
    include_attacks=['fgsm', 'pgd']
)

print(f"🛡️ Attack Suite Created:")
for i, attack in enumerate(attack_suite):
    print(f"  {i+1}. {attack.__class__.__name__}")

# Create robustness evaluator
robustness_evaluator = RobustnessEvaluator(
    attacks=attack_suite,
    certification_methods=['randomized_smoothing']
)

# Test single attack
test_model = base_model
test_model.eval()

# Generate adversarial examples with FGSM
fgsm_attack = attack_suite[0]  # First attack is FGSM
adv_examples = fgsm_attack.generate(test_model, dummy_input, dummy_targets)

# Compare clean vs adversarial predictions
with torch.no_grad():
    clean_outputs = test_model(dummy_input)
    adv_outputs = test_model(adv_examples)
    
    clean_preds = clean_outputs.argmax(dim=1)
    adv_preds = adv_outputs.argmax(dim=1)
    
    attack_success = (clean_preds != adv_preds).float().mean()

print(f"\n🛡️ Attack Results:")
print(f"Clean predictions: {clean_preds.tolist()}")
print(f"Adversarial predictions: {adv_preds.tolist()}")
print(f"Attack success rate: {attack_success:.2%}")

# Compute perturbation statistics
perturbation = adv_examples - dummy_input
max_perturbation = perturbation.abs().max().item()
avg_perturbation = perturbation.abs().mean().item()

print(f"Max perturbation: {max_perturbation:.4f}")
print(f"Average perturbation: {avg_perturbation:.4f}")

## 5. Synthetic Data Generation

Generate realistic synthetic infrared data using physics-based rendering.

In [ ]:
# Configure synthetic data generation
synthetic_config = SyntheticDataConfig(
    image_size=(128, 128),
    num_targets=(1, 2),
    target_size_range=(5, 12),
    temperature_range=(350.0, 450.0),
    background_temp=(280.0, 320.0),
    noise_level=0.03,
    atmospheric_effects=True,
    domain_randomization=True
)

print(f"🎨 Synthetic Data Configuration:")
print(f"Image size: {synthetic_config.image_size}")
print(f"Target count: {synthetic_config.num_targets}")
print(f"Target size range: {synthetic_config.target_size_range}")
print(f"Temperature range: {synthetic_config.temperature_range}K")

# Create synthetic dataset
synthetic_dataset = create_synthetic_dataset(
    config=synthetic_config,
    dataset_size=100,  # Small for demo
    use_gan=False  # Use physics-based rendering
)

print(f"\n🎨 Synthetic Dataset Created:")
print(f"Dataset size: {len(synthetic_dataset)}")

# Generate a few samples
sample_data = []
for i in range(3):
    sample = synthetic_dataset[i]
    sample_data.append(sample)
    
    print(f"\nSample {i+1}:")
    print(f"  Image shape: {sample['image'].shape}")
    print(f"  Has target: {sample['classification_target'].item()}")
    print(f"  Num targets: {len(sample['metadata'])}")
    
    if sample['metadata']:
        target_info = sample['metadata'][0]
        print(f"  Target temp: {target_info['temperature']:.1f}K")
        print(f"  Target size: {target_info['size']} pixels")

## 6. Visualization

Visualize the generated synthetic data and model predictions.

In [ ]:
# Visualize synthetic samples
fig, axes = plt.subplots(2, 3, figsize=(12, 8))
fig.suptitle('🎨 Synthetic Infrared Data Samples', fontsize=16)

for i, sample in enumerate(sample_data[:3]):
    image = sample['image'].squeeze().numpy()
    mask = sample['mask'].squeeze().numpy()
    
    # Plot image
    axes[0, i].imshow(image, cmap='hot', vmin=0, vmax=1)
    axes[0, i].set_title(f'Sample {i+1} - IR Image')
    axes[0, i].axis('off')
    
    # Plot mask
    axes[1, i].imshow(mask, cmap='gray', vmin=0, vmax=1)
    axes[1, i].set_title(f'Sample {i+1} - Target Mask')
    axes[1, i].axis('off')
    
    # Add target information
    if sample['metadata']:
        target = sample['metadata'][0]
        axes[0, i].plot(target['x'], target['y'], 'r+', markersize=10, markeredgewidth=2)

plt.tight_layout()
plt.show()

print("📊 Visualization complete!")

## 7. Integration Example

Demonstrate how these advanced features can be combined for a complete research workflow.

In [ ]:
print("🔬 Advanced Research Integration Example")
print("======================================\n")

# Step 1: Generate synthetic training data
print("Step 1: Generating synthetic training data...")
synthetic_loader = DataLoader(synthetic_dataset, batch_size=8, shuffle=True)
print(f"✓ Created DataLoader with {len(synthetic_dataset)} samples\n")

# Step 2: Create physics-informed model
print("Step 2: Creating physics-informed model...")
research_model = create_physics_informed_model(
    input_channels=1,
    num_classes=2,
    physics_laws=['atmospheric', 'infrared']
)
print(f"✓ Physics-informed model created with {len(research_model.physics_laws)} physics laws\n")

# Step 3: Setup adversarial training
print("Step 3: Setting up adversarial training...")
from irst_library.research import create_robust_trainer
robust_trainer = create_robust_trainer(
    attack_epsilon=0.05,
    training_method='trades'
)
print(f"✓ Adversarial trainer configured with TRADES method\n")

# Step 4: Demonstrate one training step
print("Step 4: Demonstrating integrated training step...")
sample_batch = next(iter(synthetic_loader))
images = sample_batch['image']
targets = sample_batch['classification_target']

# Physics-informed forward pass
physics_outputs = research_model(images)
physics_losses = research_model.compute_physics_loss(images, physics_outputs)

# Adversarial training loss
adv_losses = robust_trainer.compute_adversarial_loss(
    research_model, images, targets, physics_outputs['logits']
)

print(f"✓ Physics loss: {physics_losses['total_physics_loss']:.4f}")
print(f"✓ Adversarial loss: {adv_losses['total_loss']:.4f}")
print(f"✓ Combined advanced training step completed!\n")

# Step 5: Robustness evaluation
print("Step 5: Quick robustness evaluation...")
fgsm_adv = attack_suite[0].generate(research_model, images[:4], targets[:4])
with torch.no_grad():
    clean_acc = (research_model(images[:4])['logits'].argmax(1) == targets[:4]).float().mean()
    adv_acc = (research_model(fgsm_adv)['logits'].argmax(1) == targets[:4]).float().mean()

print(f"✓ Clean accuracy: {clean_acc:.2%}")
print(f"✓ Adversarial accuracy: {adv_acc:.2%}")
print(f"✓ Robustness gap: {(clean_acc - adv_acc):.2%}\n")

print("🎉 Advanced Research Integration Complete!")
print("\nThis demonstrates how quantum-inspired networks, physics-informed")
print("models, continual learning, adversarial robustness, and synthetic")
print("data generation can be seamlessly integrated for cutting-edge")
print("infrared small target detection research.")

## Next Steps

This tutorial covered the advanced research features in IRST Library. To dive deeper:

1. **Quantum Neural Networks**: Explore different quantum architectures and compare with classical models
2. **Physics-Informed Networks**: Implement custom physics laws for specific applications
3. **Continual Learning**: Test on sequential task scenarios with real datasets
4. **Adversarial Robustness**: Evaluate certified defenses and adaptive attacks
5. **Synthetic Data**: Scale up generation for large-scale training

### Additional Resources

- [Advanced Features Documentation](../docs/ADVANCED_FEATURES.md)
- [Research Papers Bibliography](../docs/REFERENCES.md)
- [Contributing Guidelines](../CONTRIBUTING.md)
- [Community Discussions](https://github.com/your-repo/discussions)

---

**🚀 Ready to push the boundaries of infrared target detection research!**

## 11. Active Learning for Efficient Data Annotation

Demonstrate intelligent sample selection strategies to minimize annotation costs while maximizing model performance.

In [ ]:
# Import active learning modules
from irst_library.research import (
    ActiveLearner,
    ActiveLearningConfig,
    SamplingStrategy,
    StreamingActiveLearner,
    create_active_learning_experiment,
    benchmark_active_learning_strategies
)

# Create active learning configuration
al_config = ActiveLearningConfig(
    strategy=SamplingStrategy.HYBRID,
    batch_size=32,
    budget=1000,
    diversity_weight=0.3,
    committee_size=5
)

# Initialize active learner
learner = ActiveLearner(al_config)

# Initialize with sample dataset
total_samples = 5000
initial_labeled = 100
learner.initialize_pool(total_samples, initial_labeled)

print(f"🎯 Active Learning Setup:")
print(f"  Strategy: {al_config.strategy.value}")
print(f"  Initial labeled: {len(learner.labeled_indices)}")
print(f"  Unlabeled pool: {len(learner.unlabeled_indices)}")
print(f"  Batch size: {al_config.batch_size}")

# Simulate active learning round
print("\n📊 Active Learning Metrics:")
print(f"  Total budget: {al_config.budget}")
print(f"  Diversity weight: {al_config.diversity_weight}")
print(f"  Committee size: {al_config.committee_size}")

# Create different sampling strategies for comparison
strategies = ["uncertainty", "diversity", "hybrid", "committee"]
print(f"\n🔍 Available strategies: {strategies}")

# Example of uncertainty sampling
uncertainty_config = ActiveLearningConfig(strategy=SamplingStrategy.UNCERTAINTY)
uncertainty_learner = ActiveLearner(uncertainty_config)
print(f"\n💡 Uncertainty Sampling: Selects samples with highest prediction uncertainty")

# Example of diversity sampling  
diversity_config = ActiveLearningConfig(strategy=SamplingStrategy.DIVERSITY)
diversity_learner = ActiveLearner(diversity_config)
print(f"🎨 Diversity Sampling: Selects diverse samples to improve coverage")

# Example of hybrid approach
hybrid_config = ActiveLearningConfig(
    strategy=SamplingStrategy.HYBRID,
    diversity_weight=0.4  # Balance between uncertainty and diversity
)
hybrid_learner = ActiveLearner(hybrid_config)
print(f"⚖️ Hybrid Sampling: Combines uncertainty and diversity ({hybrid_config.diversity_weight:.1f} diversity weight)")

# Streaming active learning for real-time scenarios
streaming_learner = StreamingActiveLearner(al_config)
print(f"\n🌊 Streaming Active Learning: For real-time data processing")
print(f"  Budget tracking: {streaming_learner.annotation_budget_used}/{al_config.budget}")

print("\n✅ Active Learning modules initialized successfully!")

In [ ]:
# Demonstrate advanced active learning features

# 1. Multi-objective Pareto optimization
from irst_library.research import ParetoActiveLearner

pareto_config = ActiveLearningConfig(
    strategy=SamplingStrategy.PARETO,
    pareto_objectives=["uncertainty", "diversity"]
)
pareto_learner = ParetoActiveLearner(pareto_config)

print("🎯 Pareto Active Learning:")
print(f"  Objectives: {pareto_config.pareto_objectives}")
print("  Finds optimal trade-off between multiple selection criteria")

# 2. Expected model change sampling
from irst_library.research import ExpectedModelChangeSampler

emc_config = ActiveLearningConfig(
    strategy=SamplingStrategy.EXPECTED_CHANGE,
    gradient_embedding_dim=512
)
emc_sampler = ExpectedModelChangeSampler(emc_config)

print(f"\n🔄 Expected Model Change:")
print(f"  Gradient embedding dim: {emc_config.gradient_embedding_dim}")
print("  Selects samples that would change the model the most")

# 3. Committee-based sampling with ensemble
committee_config = ActiveLearningConfig(
    strategy=SamplingStrategy.COMMITTEE,
    committee_size=7
)

print(f"\n👥 Committee Sampling:")
print(f"  Committee size: {committee_config.committee_size}")
print("  Uses ensemble disagreement to select informative samples")

# 4. Continual active learning
continual_config = ActiveLearningConfig(
    strategy=SamplingStrategy.HYBRID,
    continual_learning=True,
    memory_size=5000
)
continual_learner = ActiveLearner(continual_config)

print(f"\n🔄 Continual Active Learning:")
print(f"  Memory buffer size: {continual_config.memory_size}")
print("  Maintains memory for streaming scenarios")

# 5. Budget-aware selection
budget_config = ActiveLearningConfig(
    strategy=SamplingStrategy.UNCERTAINTY,
    budget=500,
    batch_size=16
)

print(f"\n💰 Budget-Aware Selection:")
print(f"  Total budget: {budget_config.budget}")
print(f"  Batch size: {budget_config.batch_size}")
print(f"  Max rounds: {budget_config.budget // budget_config.batch_size}")

# Demonstrate selection quality evaluation
print(f"\n📊 Selection Quality Metrics:")
print("  - Average pairwise distance (diversity)")
print("  - Minimum pairwise distance (coverage)")
print("  - Average uncertainty (informativeness)")
print("  - Selection efficiency")

# Performance tracking
print(f"\n📈 Performance Tracking:")
print("  - Learning curves")
print("  - Annotation efficiency")
print("  - Strategy comparison")
print("  - Budget utilization")

print("\n✅ Advanced active learning features demonstrated!")

In [ ]:
# Benchmarking different active learning strategies
print("🏆 Active Learning Strategy Benchmarking")

# Define strategies to compare
benchmark_strategies = [
    "uncertainty",
    "diversity", 
    "hybrid",
    "committee",
    "expected_change"
]

print(f"\nComparing strategies: {benchmark_strategies}")

# Simulate benchmark results (in real usage, this would run actual experiments)
print("\n📊 Simulated Benchmark Results:")
for strategy in benchmark_strategies:
    print(f"\n{strategy.upper()} Strategy:")
    print(f"  ├── Sample efficiency: {np.random.uniform(0.75, 0.95):.3f}")
    print(f"  ├── Diversity score: {np.random.uniform(0.6, 0.9):.3f}")
    print(f"  ├── Uncertainty coverage: {np.random.uniform(0.7, 0.95):.3f}")
    print(f"  └── Computational cost: {np.random.uniform(0.1, 0.8):.3f}")

# Learning curve visualization setup
print(f"\n📈 Learning Curve Analysis:")
print("  - X-axis: Number of labeled samples")
print("  - Y-axis: Model accuracy")
print("  - Multiple curves for different strategies")

# Strategy recommendation system
print(f"\n🎯 Strategy Recommendations:")
print("  UNCERTAINTY: Best for quick improvements on easy datasets")
print("  DIVERSITY: Best for ensuring broad coverage")
print("  HYBRID: Best balance for most scenarios")
print("  COMMITTEE: Best when computational resources available")
print("  PARETO: Best for multi-objective optimization")

# Real-world deployment considerations
print(f"\n🌍 Deployment Considerations:")
print("  ├── Annotation budget constraints")
print("  ├── Real-time vs batch processing")
print("  ├── Domain shift handling")
print("  ├── Expert annotator availability")
print("  └── Computational resource limits")

# Active learning workflow summary
print(f"\n🔄 Complete Active Learning Workflow:")
print("  1. Initialize with small labeled set")
print("  2. Train initial model")
print("  3. Select most informative unlabeled samples")
print("  4. Annotate selected samples")
print("  5. Retrain model with new data")
print("  6. Evaluate performance")
print("  7. Repeat until budget exhausted or target reached")

print("\n✅ Active learning benchmarking and analysis complete!")